In [1]:
import re
import sys
from pattern.fr import conjugate, lemma, parse, parsetree,pprint,pluralize, singularize
from collections import defaultdict
import json
import spacy
import hunspell
import fr_core_news_sm
import operator

In [3]:
#charge le modèle de langue
nlp = spacy.load('fr_core_news_sm')
h = hunspell.HunSpell('/usr/share/hunspell/fr.dic', '/usr/share/hunspell/fr.aff')

In [4]:
stopwords=["celui","celle","ceux",',','"','(',')',"s'","n'","j'","l'",'l’','leurs','toutes',"c'",'dès','etc','y','avoir','lesquelles','chacun','oui','non','ou','ne','par','plus','moins','tout','tous','faire','qui','que','pour','dans','sur','faut','il','de','des','les','la','le','du','à','et','on','un','une','au','aux','son','sa','en','avec','pas','']
#ajoute à la liste des stopwords ceux trouvés sur ce site
#https://www.ranks.nl/stopwords/french
stop=open('stopwords.txt', 'r',encoding='utf-8')
for line in stop :
    line=line.strip('\ufeff')
    stopwords.append(line.strip('\n'))
stop.close()
#print(stopwords)

In [3]:
#renvoie liste de tuples (mot,tag)
#query = parse(txt)
def convert_tag_format(query): 
    word = query.split(' ')
    postag = [(x.split('/')[0], x.split('/')[1]) for x in word]
    return postag 

In [5]:
#renvoie liste de tuples (mot,chunk, chunkbis)
def convert_chunk_format(query): 
    word = query.split(' ')
    postag = [(x.split('/')[0], x.split('/')[2],x.split('/')[3]) for x in word]
    return postag 

In [6]:
#renvoie liste de tuples (mot,tag) + phrase taggée brut (ex : Il/PRP/B-NP/O faut/VB/B-VP/O)
#mot/tag/chunk/O tag (= outside) means that the word is not part of a chunk.
def get_pos_tags(text): 
    tagged_sent = parse(text)
    return convert_tag_format(tagged_sent), tagged_sent

In [7]:
def normalise(word):
    word = word.lower()
    return word

In [8]:
#conditions for acceptable word: length, stopword
def acceptable_word(word):
    accepted = bool(2 <= len(word) <= 40
        and word.lower() not in stopwords and lemma(word.lower()) not in stopwords)
    return accepted

In [9]:
#return ex : {'NP': ['il', 'les abattoirs'], 'VP': ['faut fermer']}
#extract entity from BIO encoding 
def extract_entity(filetext):
    last_entity = '' 
    last_tag = '' 
    mention2entities = {} 
    for line in filetext.split('\n'): 
        line = line.strip() 
        if line == '': 
            continue
        line_split = line.split('\t')
        if re.search('B-', line_split[1]): 
            if last_entity != '': 
                if not last_tag in mention2entities:
                    mention2entities[last_tag] = [] 
                mention2entities[last_tag].append(last_entity.strip())
            last_entity = line_split[0] + ' '
            last_tag = line_split[1][2:] 
        elif re.search('I-', line_split[1]): 
            last_entity += line_split[0] + ' '
    if last_entity != '': 
        if not last_tag in mention2entities:
            mention2entities[last_tag] = [] 
        mention2entities[last_tag].append(last_entity.strip())
    return mention2entities

In [10]:
#phrase2consider=liste (ex : ['NP', 'ADJP'])
#tagged_sent = phrase taggée brut (ex : Il/PRP/B-NP/O faut/VB/B-VP/O)
#mention2entities = dico (ex : {'NP': ['il', 'les abattoirs'], 'VP': ['faut fermer']})

#renvoie les entités présentes pour des types de phrase/groupe donnés, ex VP et NP etc
def get_entities_from_phrase(tagged_sent, phrase2consider): 
    word = tagged_sent.split(' ')
    bio_tags = [normalise(x.split('/')[0])+ '\t'+ x.split('/')[2] for x in word]
    bio_text = '\n'.join(bio_tags)
    mention2entities = extract_entity(bio_text)
    
    ## strip off unacceptable words 
    _mention2entities = {} 
    for mention in mention2entities: 
        if not mention in phrase2consider: 
            continue
        _mention2entities[mention] = [] 
        for entity in mention2entities[mention]: 
            _entity = ' '.join([word for word in entity.split(' ') if acceptable_word(word)]).strip()
            if _entity != '': 
                _mention2entities[mention].append(_entity)
    entities = []
    for mention in _mention2entities: 
        entities.extend(_mention2entities[mention])
    #print(_mention2entities)
    #print(mention2entities)
    return entities

In [11]:
#renvoie liste de keywords, ici considéré par défaut comme entités dans NP et ADJP
def get_keywords(text, phrase2consider=['NP', 'ADJP']): 
    toRemove=[]
    try:
        postoks, tagged_sent = get_pos_tags(text)
        entities = get_entities_from_phrase(tagged_sent, phrase2consider)
        doc=nlp(text)
        if (phrase2consider==['NP', 'ADJP']) :
            for i in range(2,len(doc)) :
                tok=doc[i]
                beforetok=(doc[i-1])
                if i>2 : 
                    before2tok=(doc[i-2])
                if(tok.dep_=='nmod' and beforetok.dep_=='case' and tok.pos_=='NOUN' and tok.head.pos_=='NOUN'):
                    add=tok.head.text+' '+beforetok.text+' '+tok.text
                    entities.append(add)
                    ##toRemove.append(tok.text)
                    ##toRemove.append(tok.head.text)
                    entities.append(tok.head.text)
                    entities.append(tok.text)
                if(tok.dep_=='nmod' and before2tok.dep_=='case' and tok.pos_=='NOUN' and tok.head.pos_=='NOUN' and beforetok.pos_=='ADJ'):
                    add=tok.head.text+' '+before2tok.text+' '+beforetok.text+' '+tok.text
                    entities.append(add)
                    ##toRemove.append(tok.text)
                    ##toRemove.append(tok.head.text)
                    entities.append(tok.text)
                    entities.append(tok.head.text)
                if(tok.dep_=='appos' and tok.head.pos_=='NOUN') :
                    add=tok.head.text+' '+tok.text
                    entities.append(add)
                    ##toRemove.append(tok.text)
                    ##toRemove.append(tok.head.text)
                    entities.append(tok.text)
                    entities.append(tok.head.text)
                if(tok.pos_=='NOUN' and tok.dep_=='fixed' and beforetok.pos_=='ADP' and beforetok.dep_=='advmod' and beforetok.head.pos_=='NOUN') :
                    add=before2tok.text+' '+beforetok.text+' '+tok.text
                    entities.append(add)
                    ##toRemove.append(tok.text)
                    ##toRemove.append(before2tok.text)
                    entities.append(tok.text)
                    entities.append(before2tok.text)
                if(tok.pos_=='NUM' and before2tok.pos_=='NOUN' and beforetok.pos_=='ADP' and beforetok.dep_=='case' and beforetok.head==tok):
                    add=before2tok.text+' '+beforetok.text+' '+tok.text
                    entities.append(add)
                    ##toRemove.append(tok.text)
                    entities.append(tok.text)
                if(tok.pos_=='NOUN' and beforetok.pos_=='DET' and before2tok.pos_=='NOUN') :
                    add=before2tok.text+' '+beforetok.text+' '+tok.text
                    entities.append(add)
                    entities.append(tok.text)
                    entities.append(before2tok.text)
                if( tok.text not in stopwords and tok.pos_=='ADJ' and beforetok.pos_=='ADJ' and before2tok.pos_=='NOUN' and tok.head==before2tok and beforetok.head==before2tok and tok.dep_=='amod' and beforetok.dep_=='amod') : 
                    add=before2tok.text+' '+beforetok.text+' '+tok.text
                    entities.append(add)
                #(tok.head.pos_=='NOUN' and tok.dep_=='amod') or (tok.pos_=='NOUN' and tok.dep_=='amod')
                if(tok.head.pos_=='NOUN' and tok.dep_=='amod' and tok.text not in stopwords ) :
                    if(tok.i < tok.head.i) :
                        toAdd=tok.text+' '+tok.head.text
                    else :
                        toAdd=tok.head.text+' '+tok.text
                    if(toAdd not in entities) :
                        entities.append(toAdd)
                    ##toRemove.append(tok.text)
                    ##toRemove.append(tok.head.text)
                    entities.append(tok.text)
                    entities.append(tok.head.text)
            for elt in toRemove :
                if elt in entities :
                    entities.remove(elt)
    except: 
        return []
    return list(set(entities))

In [12]:
#modifier ce que retourne get_keywords
entities=get_keywords("Il faut que les produits locaux soient au même prix que les produits importés, voire moins chers, pour qu'ils soient accessibles à tous",['NP', 'ADJP'])
lemme2words=defaultdict(lambda:defaultdict(int))
keywords=[]
for entity in entities : 
    for word in entity.split():
        lemme=h.stem(word)[0].decode('utf-8')
        lemme2words[lemme][word]+=1
        keywords.append(lemme)
print(lemme2words)
for i in range(len(keywords)) : 
    keywords[i]=max(dict(lemme2words[keywords[i]]).items(), key=operator.itemgetter(1))[0]
    print(keywords[i])

defaultdict(<function <lambda> at 0x7fc51d353f28>, {})


In [13]:
#Si on effectue à partir d'un dico enregistré (json)
print("Nom du fichier où se trouve le dico d'annotations")
filename=input()
json_file = open(filename+'.json',encoding='utf-8')
jsontxt = json_file.read()
jsondicoannot=json.loads(jsontxt)
json_file.close()

Nom du fichier où se trouve le dico d'annotations
json_annots


FileNotFoundError: [Errno 2] No such file or directory: 'json_annots.json'

In [14]:
annotspattern=defaultdict(list)
annotslemme=defaultdict(list)
lemme2words=defaultdict(lambda:defaultdict(int))
for k,v in jsondicoannot.items() :
    keywords=[]
    annotspattern[k]+=get_keywords(k,['NP', 'ADJP'])
    for entity in annotspattern[k] : 
        lemme=""
        for word in entity.split():
            try :
                lemme=lemme+h.stem(word)[0].decode('utf-8')+" "
            except IndexError :
                pass;
        lemme2words[lemme.strip()][entity]+=1
        keywords.append(lemme.strip())
    print("ANNOTLEMME")
    annotslemme[k]=keywords
    print(annotslemme[k])
    print(k)
    print("pattern :")
    print(annotspattern[k])
"""for i in range(len(keywords)) : 
    keywords[i]=max(dict(lemme2words[keywords[i]]).items(), key=operator.itemgetter(1))[0]"""
for k,v in jsondicoannot.items() :
    for i in range(len(annotspattern[k])) :
        annotspattern[k][i]=max(dict(lemme2words[annotslemme[k][i]]).items(), key=operator.itemgetter(1))[0]
    print(k)
    print("pattern :")
    print(annotspattern[k])
    """print(k)
    print("pattern :")
    print(annotspattern[k])
    print("annots manuelles :")
    print(jsondicoannot[k])
    print("communs : ")
    print(set(annotspattern[k]).intersection(set(jsondicoannot[k])))
    print("\n")"""

NameError: name 'jsondicoannot' is not defined

In [64]:
TP=0
rapdenum=0
precisiondenum=0
for k,v in jsondicoannot.items() :
    print(k)
    print(list(set(jsondicoannot[k])))
    print(list(set(annotspattern[k])))
    print()
    print("Rappel : "+str(len(set(annotspattern[k]).intersection(set(jsondicoannot[k]))))+'/'+str(len(jsondicoannot[k])))
    TP+=len(set(annotspattern[k]).intersection(set(jsondicoannot[k])))
    rapdenum+=len(jsondicoannot[k])
    print("Précision : "+str(len(set(annotspattern[k]).intersection(set(jsondicoannot[k]))))+'/'+str(len(annotspattern[k])))
    precisiondenum+=len(annotspattern[k])
print(TP/precisiondenum)
print(TP/rapdenum)

Il faut aider à développer une agriculture raisonnée et à favoriser un élevage de qualité et sain
['élevage de qualité', 'agriculture raisonnée']
['élevage', 'sain', 'qualité', 'agriculture raisonnée']

Rappel : 1/2
Précision : 1/4
Il faut faire revenir les petits commerçants au cœur des villes et villages et privilégier les circuits courts et le bio
['petits commerçants', 'circuits courts', 'villes et villages', 'bio']
['petits commerçants', 'villes villages', 'cœur', 'circuits courts', 'bio']

Rappel : 3/4
Précision : 3/5
Il faut développer la vente en vrac 
['vente en vrac']
['vrac', 'vente']

Rappel : 0/1
Précision : 0/2
Il faut manger et créer français
['français']
['français']

Rappel : 1/1
Précision : 1/1
Il faut continuer la suppression des substances controversées / additifs dans tous les produits alimentaires
['additifs', 'suppression des substances controversées', 'produits alimentaires']
['suppression', 'substances', 'produits alimentaires', 'additifs']

Rappel : 2/3
Précis

In [26]:
#keywords uniquement VP
for k,v in jsondicoannot.items() :
    print(k)
    print("pattern :")
    print(set(get_keywords(k,['VP'])))
    print('\n')

Il faut une information claire concernant la qualité, l'origine des produits et surtout la consommation des produits de saison
pattern :
set()


Il faut prendre du temps pour préparer ses repas, ne jamais acheter du tout fait.
pattern :
set()


Il faut acheter directement au producteur (agriculteur) pour augmenter son revenu mensuel
pattern :
set()


Il ne faut plus d'élevages industriels, mais des élevages ou les animaux sont traités humainement
pattern :
set()


Il faut une taxe écologique pour les produits qui génèrent trop d'effets de serre, de pollution des sols, de l'eau etc.
pattern :
set()


Il faut apprendre à distinguer les calories des composants sains (glucides, protéines …) et les calories des sucres, graisses saturées, etc
pattern :
set()


Il faut beaucoup plus de produits BIO, notamment légumes et fruits frais dans les magasin U
pattern :
set()


Il faut que les produits locaux soient au même prix que les produits importés, voire moins chers, pour qu'ils soient accessib

In [15]:
s = parsetree("Il faut manger des produits de saison et qui viennent de circuit court.", relations=True, lemmata=True)
for sentence in s : 
    for chunk in sentence.chunks : 
        print(chunk.type, [(w.string, w.type) for w in chunk.words], chunk.pnp)

NP [('Il', 'PRP')] None
VP [('faut', 'VB'), ('manger', 'VB')] None
PP [('des', 'IN')] des produits
NP [('produits', 'NNS')] des produits
PP [('de', 'IN')] de saison
NP [('saison', 'NN')] de saison
VP [('viennent', 'VB')] None
PP [('de', 'IN')] de circuit court
NP [('circuit', 'NN'), ('court', 'RB')] de circuit court


In [28]:
pprint(parse('Il faut manger des produits de saison et qui viennent de circuit court', relations=True, lemmata=True))

Pretty printing has been turned OFF


In [16]:
s = parsetree("Il faut éviter d'acheter des produits transformés et cuisiner avec des produits de base", relations=True, lemmata=True)
for sentence in s : 
    for chunk in sentence.chunks : 
        print(chunk.type, [(w.string, w.type) for w in chunk.words], chunk.pnp)

NP [('Il', 'PRP')] None
VP [('faut', 'VB'), ('éviter', 'VB')] None
PP [("d'", 'IN')] None
VP [('acheter', 'VB')] None
PP [('des', 'IN')] des produits transformés
NP [('produits', 'NNS')] des produits transformés
VP [('transformés', 'VBN')] des produits transformés
VP [('cuisiner', 'VB')] None
PP [('avec', 'IN'), ('des', 'IN')] avec des produits
NP [('produits', 'NNS')] avec des produits
PP [('de', 'IN')] de base
NP [('base', 'NN')] de base
